# DOCK6 Mutation Trans Effect on Transcriptomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "DOCK6"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Transcriptomics

### Generate interacting protein list

Make a call to cptac.utils to get the interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene.

In [6]:
# Use get interacting proteins method to generate list of interacting proteins
interacting_proteins = al.get_interacting_proteins(gene)

# Show interacting protein list
print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
DOCK1
DHX8
DOCK6
MGRN1
ARHGAP31
CRK
SWSAP1
DHX37
ELMO1
RHBDF2
DDHD1
LRCH3
GRAP2
TMEM205
RAC1
SNRNP25
EOGT
MOB1A
CDC42
DOCK8
ENDOU
AKT1
KANK2
CCDC159
C19orf80
NPRL3
TADA2A
UBC
CRLF3
DOCK7
MOB1B
XPO1
CLU
MYH6
RAB2A
LRP8
REEP5
SLC33A1
RNASEH2A
RAB32
KIF7
Smad3
Msn
Rock1
Kifc1
Cep135


### Test for significant comparisons in any of interacting proteins

In [7]:
# Create dataframe in order to do comparisons with wrap_ttest
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, flatten=True)
protdf = protdf[protdf.Sample_Status == 'Tumor'] # drop Normal samples

# Create the binary valued column needed to do the comparison
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

# Format the dataframe correctly
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


# Make list of columns to be compared using t-tests
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

# Call wrap_ttest, pass in formatted dataframe
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

# Print results, if anything significant was found
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)
        

Doing t-test comparisons

No significant comparisons.


# All Proteins: Transcriptomics 

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten=True) # flatten for specific column names
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create binary column 
    protdf['Label'] = np.where(
                protdf[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  DOCK6


Doing t-test comparisons

                                           Comparison       P_Value
0        RNU6-1304P_transcriptomics_ENSG00000223306.2  9.536445e-16
1           RPSAP64_transcriptomics_ENSG00000227721.1  8.820364e-10
2     RP11-285A15.1_transcriptomics_ENSG00000250191.1  9.031615e-10
3         RNU6-752P_transcriptomics_ENSG00000252185.1  9.031615e-10
4          TNFSF11_transcriptomics_ENSG00000120659.13  7.726449e-09
5           SCML2P1_transcriptomics_ENSG00000264127.1  1.565341e-08
6           MIR6846_transcriptomics_ENSG00000277888.1  2.606630e-08
7           UBE2SP1_transcriptomics_ENSG00000233966.1  3.147624e-08
8   RP3-336K20__B.2_transcriptomics_ENSG00000236512.1  1.230388e-07
9     RP11-254F19.2_transcriptomics_ENSG00000260859.1  1.441931e-07
10     RP5-1011O1.3_transcriptomics_ENSG00000236069.1  1.619497e-07
11             MMP9_transcriptomics_ENSG00000100985.7  1.840137e-07
12      CTB-31O20.9_transcriptomics_ENSG00000267232.1  1.849078e-07
13       ABHD15-AS1_tr

# Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,DOCK6,RNU6-1304P_transcriptomics_ENSG00000223306.2,False,9.536445e-16
1,Gbm,DOCK6,RPSAP64_transcriptomics_ENSG00000227721.1,False,8.820364e-10
2,Gbm,DOCK6,RP11-285A15.1_transcriptomics_ENSG00000250191.1,False,9.031615e-10
3,Gbm,DOCK6,RNU6-752P_transcriptomics_ENSG00000252185.1,False,9.031615e-10
4,Gbm,DOCK6,TNFSF11_transcriptomics_ENSG00000120659.13,False,7.726449e-09
5,Gbm,DOCK6,SCML2P1_transcriptomics_ENSG00000264127.1,False,1.565341e-08
6,Gbm,DOCK6,MIR6846_transcriptomics_ENSG00000277888.1,False,2.606630e-08
7,Gbm,DOCK6,UBE2SP1_transcriptomics_ENSG00000233966.1,False,3.147624e-08
8,Gbm,DOCK6,RP3-336K20__B.2_transcriptomics_ENSG00000236512.1,False,1.230388e-07
9,Gbm,DOCK6,RP11-254F19.2_transcriptomics_ENSG00000260859.1,False,1.441931e-07


### Write Significant Comparisons (if any) to Shared CSV file

In [10]:
'''
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)'''

"\nexisting_results = pd.read_csv(gene+'_Trans_Results.csv')\n\nupdated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)\n\nupdated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)"